In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

## Exploratory Analysis

In [ ]:
df = pd.read_csv('/home/alvaro/tf_templates/DATA/cancer_classification.csv')
df.info()

In [ ]:
df.describe().transpose()

In [ ]:
sns.countplot(data=df, x='benign_0__mal_1');

In [ ]:
sns.heatmap(df.corr());

In [ ]:
df.corr()['benign_0__mal_1'].sort_values()

In [ ]:
df.corr()['benign_0__mal_1'][:-1].sort_values().plot(kind='bar');

## Scaling and Traing-Test Split

In [ ]:
X = df.drop('benign_0__mal_1', axis=1).values
y = df['benign_0__mal_1'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Creating the Model and Training

In [ ]:
X_train.shape

In [ ]:
X_test.shape

### Example 1: Overfitting

In [ ]:
model = Sequential()

model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=128, epochs=600)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot();

### Example 2: Early Stop

In [ ]:
model = Sequential()

model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=15, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=25)

In [ ]:
model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), callbacks=[early_stop], batch_size=128, epochs=600)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot();

### Example 3: Dropout and Best Weights

In [ ]:
model = Sequential()

model.add(Dense(units=30, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=15, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=True)

In [ ]:
model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), callbacks=[early_stop], batch_size=128, epochs=600)

## Evaluation

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot();

In [ ]:
# binary classification
predictions = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
print(confusion_matrix(y_test, predictions))